## 🛰️ Imágenes de Cobertura y Uso del Suelo de MapBiomas Colombia – Colección 2 (1985–2023)

La Colección 2 de MapBiomas Colombia (1985–2023) ofrece mapas anuales de cobertura y uso del suelo generados a partir de series temporales de imágenes satelitales. Estos datos están disponibles de forma pública como un **asset en Google Earth Engine (GEE)**.

### 📦 Asset disponible en GEE
- **asset:** projects/mapbiomas-public/assets/colombia/collection2/mapbiomas_colombia_collection2_integration_v1

Este asset corresponde a una **imagen multibanda**, donde cada banda representa un año específico. Por ejemplo:

- `classification_1985`
- `classification_2001`
- `classification_2022`
- `classification_2023`

---

## 🧪 Requisitos para usar en JupyterLab desde Docker

### 1. Instalar librerías necesarias dentro del contenedor

En una celda de Jupyter o desde la terminal del contenedor, es necesario ejecutar:

```bash
pip install earthengine-api geemap jupyter-leaflet


| 📦 Paquete           | 📝 Función                                                                 |
|----------------------|---------------------------------------------------------------------------|
| `earthengine-api`    | Para acceder al catálogo de Google Earth Engine (GEE)                     |
| `geemap`             | Para visualizar datos geoespaciales en mapas interactivos                |
| `jupyter-leaflet`    | Para mostrar mapas dentro de JupyterLab usando `geemap`                  |

### 2. 🔐 Autenticación en Google Earth Engine

Desde el notebook, ejecuta el siguiente código para autenticar tu cuenta de Google Earth Engine:

```python
import ee

# Solo es necesario la primera vez
ee.Authenticate()
ee.Initialize()
```
Esto abrirá un enlace en tu navegador donde deberás iniciar sesión con tu cuenta de Google asociada a Earth Engine.
Luego, copia el código de verificación que te aparece y pégalo en el campo solicitado del notebook.

✅ Este paso solo es necesario una vez por entorno. Si cierras el contenedor o cambias de entorno, puede que debas volver a autenticarte.


In [1]:
import sys
sys.path.append("../src")
import data_preprocessing
data_preprocessing.authenticate_earth_engine()

✅Auteticación y conexión con Google Earth Engine exitosa


In [2]:
imagen = data_preprocessing.load_mapbiomas_asset()
print (imagen.bandNames().getInfo())

['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_2023']


# ✂️ Recorte y reproyección de capas vectoriales (SHP) a EPSG:9377
Este flujo permite recortar y reproyectar capas como departamentos, resguardos indígenas y áreas protegidas.

## 1. 📁 Cargar los archivos .shp

In [3]:
dept, reg, resg, runap = data_preprocessing.load_geospatial_layers("DATOS")

✅Finalizó la función load_geospatial_layers


## 2. 🧭 Asegurar proyecciones y recortar por Colombia

In [4]:
# Reproyectar las capas
dept_9377, resg_9377, runap_9377, reg_9377 = data_preprocessing.reproject_layers(dept, resg, runap, reg)

✅Finalizó la función reproject_layers


## 3. ✂️ Recortar capas al área de estudio

In [5]:
# Recortar las capas a la región amazónica
dept_ama, resg_ama, runap_ama, region_ama = data_preprocessing.clip_layers_to_region(dept_9377, resg_9377, runap_9377, reg_9377)

✅Finalizó la función clip_layers_to_region


## 4. ✅ 1. Guardar las capas recortadas y reproyectadas

In [6]:
# Guardar las capas recortadas y reproyectadas
data_preprocessing.save_layers(dept_ama, resg_ama, runap_ama, region_ama, "SALIDAS")

✅ Capas guardadas correctamente en: SALIDAS
